# This is my note of study SqueezeDet 

## Importing the libraries

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import cv2
from datetime import datetime
import os.path
import sys
import time

import numpy as np
from six.moves import xrange
import tensorflow as tf
import threading

from config import *
from dataset import pascal_voc, kitti
from utils.util import sparse_to_dense, bgr_to_rgb, bbox_transform
from nets import *

## Determine if train_dir is exist, if it is exist will delete it.
## Then make a file for training

In [ ]:
def main(argv=None): 
    if tf.gfile.Exists(FLAGS.train_dir): #
        tf.gfile.DeleteRecursively(FLAGS.train_dir)

    tf.gfile.MakeDirs(FLAGS.train_dir)
    train()

<table><tr><td bgcolor=orange > <font face="purisa" size=8 color='0000ff'>
 Extended learning
</font></td></tr></table>
<br>
<table><tr><td bgcolor=white > <font face="sawasdee" size=8 color='0000ff'>
 tf.gfile.DeleeRecursively & tf.gfile.MakeDirs
</font></td></tr></table>
<br>



<font face="微软雅黑" size=6 color='0000ff'> 1. Let us know "tf.gfile"<br> </font>
<br>
 
<font face="微软雅黑" size=4 color='000000'>
The main roles of the tf.gfile module are:
    <br>
    *:To provide an API that is close to Python's file objects <br>
    *:To provide an implementation based on TensorFlow's C++ filesystem API
<br> 
</font>

<br>

<font face="微软雅黑" size=6 color='0000ff'>2. Let us know "tf.gfile.DeleeRecursively"  <br></font>
<br>

<font face="微软雅黑" size=4 color='000000'>
tf.gfile.DeleteRecursively(dirname)

Defined in tensorflow/python/lib/io/file_io.py.

Deletes everything under dirname recursively.
Args:

    *dirname: string, a path to a directory

Raises:

    *errors.OpError: If the operation fails.
 </font>
<br>   
    
    
<font face="微软雅黑" size=6 color='0000ff'> 3. Let us know "tf.gfile.MakeDirs" </font><br>  

<font face="微软雅黑" size=4 color='#000000'>    
tf.gfile.MakeDirs(dirname)

Defined in tensorflow/python/lib/io/file_io.py.

Creates a directory and all parent/intermediate directories.

It succeeds if dirname already exists and is writable.
Args:

    *dirname: string, name of the directory to be created

Raises:

    *errors.OpError: If the operation fails.

 </font>
<br>  

## Setting 'CUDA_VISIBLE_DEVICES' to use  witch GPU for trianing

In [ ]:
def train():
  """Train SqueezeDet model"""
  assert FLAGS.dataset == 'KITTI', \
      'Currently only support KITTI dataset'

  os.environ['CUDA_VISIBLE_DEVICES'] = FLAGS.gpu

<table><tr><td bgcolor=orange > <font face="purisa" size=8 color='0000ff'>
 Extended learning
</font></td></tr></table>
<br>
<table><tr><td bgcolor=white > <font face="sawasdee" size=8 color='0000ff'>
 assert
</font></td></tr></table>
<br>
## Python's assert statement helps you find bugs more quickly and with less pain. This note has some suggestions on good ways to use it. 

## For example

In [40]:
dataset = 'dd'
assert dataset == 'KITTI', \
      'Currently only support KITTI dataset'

AssertionError: Currently only support KITTI dataset

## Define operations and tensors use tf.Graph().as_default().

In [ ]:
with tf.Graph().as_default():

<table><tr><td bgcolor=orange > <font face="purisa" size=8 color='0000ff'>
 Extended learning
</font></td></tr></table>
<br>
<table><tr><td bgcolor=white > <font face="sawasdee" size=8 color='0000ff'>
 with
</font></td></tr></table>
<br>
有一些任务，可能事先需要设置，事后做清理工作。对于这种场景，Python的with语句提供了一种非常方便的处理方式。一个很好的例子是文件处理，你需要获取一个文件句柄，从文件中读取数据，然后关闭文件句柄。
如果不用with语句，代码如下：

file = open("/tmp/foo.txt")
data = file.read()
file.close()

这里有两个问题。一是可能忘记关闭文件句柄；二是文件读取数据发生异常，没有进行任何处理。下面是处理异常的加强版本：

file = open("/tmp/foo.txt")
try:
    data = file.read()
finally:
    file.close()

虽然这段代码运行良好，但是太冗长了。这时候就是with一展身手的时候了。除了有更优雅的语法，with还可以很好的处理上下文环境产生的异常。下面是with版本的代码：

with open("/tmp/foo.txt") as file:
    data = file.read()

## Specify the parameters to tune, such as image width, image height, batch size, weight decay...
## Specify the Layer imformation of SqueezeDet model
## kitti_squeezeDet_config() from ./scr/config/kitti_squeezeDet_config.py

### def kitti_squeezeDet_config():
###  """Specify the parameters to tune below."""
###  mc                       = base_model_config('KITTI')

###  mc.IMAGE_WIDTH           = 1248
###  mc.IMAGE_HEIGHT          = 384
###  mc.BATCH_SIZE            = 20

###  mc.WEIGHT_DECAY          = 0.0001
###  mc.LEARNING_RATE         = 0.01
###  mc.DECAY_STEPS           = 10000
###  mc.MAX_GRAD_NORM         = 1.0
###  mc.MOMENTUM              = 0.9
###  mc.LR_DECAY_FACTOR       = 0.5

###   mc.LOSS_COEF_BBOX        = 5.0
###   mc.LOSS_COEF_CONF_POS    = 75.0
###   mc### .LOSS_COEF_CONF_NEG    = 100.0
###   mc.LOS### S_COEF_CLASS       = 1.0

###   mc.PLOT_PROB_THRESH      = 0.4
###   mc.NMS_THRESH            = 0.4
###   mc.PROB_THRESH           = 0.005
###   mc.TOP_N_DETECTION       = 64

###   mc.DATA_AUGMENTATION     = True
###   mc.DRIFT_X               = 150
###   mc.DRIFT_Y               = 100
###   mc.EXCLUDE_HARD_EXAMPLES = False

###   mc.ANCHOR_BOX            = set_anchors(mc)
###   mc.ANCHORS               = len(mc.ANCHOR_BOX)
###   mc.ANCHOR_PER_GRID       = 9

###   return mc

# classes and classes names form base_mode_config('KITTI')

## SqueezeDet() from ./scr/nets/SqueezeDet

###   self.caffemodel_weight = joblib.load(mc.PRETRAINED_MODEL_PATH)

###    conv1 = self._conv_layer(
###        'conv1', self.image_input, filters=64, size=3, stride=2,
###        padding='SAME', freeze=True)
###    pool1 = self._pooling_layer(
###        'pool1', conv1, size=3, stride=2, padding='SAME')

###    fire2 = self._fire_layer(
###        'fire2', pool1, s1x1=16, e1x1=64, e3x3=64, freeze=False)
###    fire3 = self._fire_layer(
###        'fire3', fire2, s1x1=16, e1x1=64, e3x3=64, freeze=False)



In [3]:
    ## elif FLAGS.net == 'squeezeDet':

mc = kitti_squeezeDet_config()
print(mc)
mc.IS_TRAINING = True
mc.PRETRAINED_MODEL_PATH = '../squeezeDet-master/data/SqueezeNet/squeezenet_v1.1.pkl'#FLAGS.pretrained_model_path
model = SqueezeDet(mc)


{'NMS_THRESH': 0.4, 'LOSS_COEF_CONF_NEG': 100.0, 'KEEP_PROB': 0.5, 'IMAGE_WIDTH': 1248, 'NUM_THREAD': 4, 'LOSS_COEF_CONF_POS': 75.0, 'GRID_POOL_HEIGHT': 7, 'PLOT_PROB_THRESH': 0.4, 'TOP_N_DETECTION': 64, 'LOSS_COEF_CLASS': 1.0, 'BGR_MEANS': array([[[ 103.939,  116.779,  123.68 ]]]), 'PROB_THRESH': 0.005, 'BATCH_SIZE': 20, 'DATASET': 'KITTI', 'DEBUG_MODE': False, 'MAX_GRAD_NORM': 1.0, 'EXP_THRESH': 1.0, 'DRIFT_X': 150, 'LEARNING_RATE': 0.01, 'CLASS_NAMES': ['car', 'pedestrian', 'cyclist'], 'LEAKY_COEF': 0.1, 'EXCLUDE_HARD_EXAMPLES': False, 'QUEUE_CAPACITY': 100, 'DATA_AUGMENTATION': True, 'GRID_POOL_WIDTH': 7, 'PRETRAINED_MODEL_PATH': '', 'CLASSES': 3, 'IS_TRAINING': False, 'ANCHOR_BOX': array([[   15.79746835,    15.36      ,    36.        ,    37.        ],
       [   15.79746835,    15.36      ,   366.        ,   174.        ],
       [   15.79746835,    15.36      ,   115.        ,    59.        ],
       ..., 
       [ 1232.20253165,   368.64      ,   224.        ,   108.        ],


imdb = kitti(FLAGS.image_set, FLAGS.data_path, mc)

<table><tr><td bgcolor=orange > <font face="purisa" size=8 color='0000ff'>
 Extended learning
</font></td></tr></table>
<br>
<table><tr><td bgcolor=white > <font face="sawasdee" size=6 color='0000ff'>
Read the names and parameters of layers from .ckpt
</font></td></tr></table>
<br>
## Sometimes we have a .cpkt file, and we want to get the names and parameters of layers.

In [3]:
import os
import numpy as np
from tensorflow.python import pywrap_tensorflow
checkpoint_path="../squeezeDet-master/data/model_checkpoints/squeezeDet/model.ckpt-87000"
# print(path.getcwdu())
print(checkpoint_path)
#read data from checkpoint file
reader=pywrap_tensorflow.NewCheckpointReader(checkpoint_path)
var_to_shape_map=reader.get_variable_to_shape_map()
data_print=np.array([])
for key in var_to_shape_map:
    print('tensor_name',key)
    ckpt_data=np.array(reader.get_tensor(key))#cast list to np arrary
    ckpt_data=ckpt_data.flatten()#flatten list
    data_print=np.append(data_print,ckpt_data,axis=0)
 
 
print(data_print,data_print.shape,np.max(data_print),np.min(data_print),np.mean(data_print))

../squeezeDet-master/data/model_checkpoints/squeezeDet/model.ckpt-87000
tensor_name fire4/expand1x1/biases
tensor_name fire9/expand1x1/kernels/Momentum
tensor_name fire6/expand1x1/biases
tensor_name fire4/expand3x3/kernels
tensor_name fire11/expand3x3/kernels/Momentum
tensor_name fire6/expand3x3/biases/Momentum
tensor_name fire2/squeeze1x1/biases
tensor_name fire2/squeeze1x1/kernels
tensor_name fire11/expand3x3/kernels
tensor_name fire5/expand1x1/kernels/Momentum
tensor_name fire10/expand3x3/biases
tensor_name fire7/squeeze1x1/kernels/Momentum
tensor_name fire4/squeeze1x1/biases
tensor_name fire2/expand1x1/kernels/Momentum
tensor_name fire8/squeeze1x1/biases
tensor_name fire6/expand1x1/kernels
tensor_name fire4/expand1x1/kernels
tensor_name fire11/expand3x3/biases/Momentum
tensor_name fire10/expand1x1/kernels
tensor_name fire4/squeeze1x1/kernels
tensor_name fire4/expand3x3/kernels/Momentum
tensor_name fire4/squeeze1x1/biases/Momentum
tensor_name fire11/expand1x1/kernels
tensor_name fir

In [7]:
parameter_fire4 = reader.get_tensor("conv12/biases")
print(parameter_fire4.shape)
print(parameter_fire4[:,])

(72,)
[  2.67206039e-02   3.06304753e-01  -3.33028406e-01   6.52096748e-01
  -2.89292663e-01  -3.62792015e-01   1.01041019e+00  -7.65033066e-01
  -2.45314330e-01   6.69764757e-01  -6.41299248e-01  -2.83657033e-02
  -3.77765059e-01   5.24263263e-01  -1.46511480e-01   1.41501911e-02
  -1.65120184e-01   1.50972545e-01   7.24290848e-01  -4.63925958e-01
  -2.60360301e-01  -3.25740665e-01   2.44966179e-01   8.07715505e-02
   1.16953218e+00  -8.97056758e-01  -2.72386789e-01  -8.33941102e-02
  -1.32141984e+00  -9.77066934e-01  -1.27273750e+00  -3.48524332e-01
  -1.08383143e+00  -1.42020071e+00  -8.32717836e-01  -5.30676067e-01
  -2.60286499e-02  -3.37756872e-02  -4.75859255e-01  -2.89946273e-02
   2.20363820e-03   2.84101702e-02   6.77693561e-02   1.16262794e-01
   7.13630253e-03  -1.20983338e-02  -1.49468407e-01   1.44799636e-03
  -1.53781287e-03   1.75991151e-02  -7.02554360e-02   9.85950753e-02
  -3.72283310e-02   1.90018322e-02  -2.42904842e-01   3.53472680e-02
   1.04729421e-02   6.301566

## Print the values of the classification and the anchor

In [3]:
print("Anchor number:", mc.ANCHOR_PER_GRID)
print("Classes number:", mc.CLASSES)
print("Classes number names", mc.CLASS_NAMES)
num_output = mc.ANCHOR_PER_GRID * (mc.CLASSES + 1 + 4)

Anchor number: 9
Classes number: 3
Classes number names ['car', 'pedestrian', 'cyclist']


## Detail of kitti function

In [4]:
import cv2
import os 
import numpy as np
import subprocess

from dataset.imdb import imdb
from utils.util import bbox_transform_inv, batch_iou

In [5]:

class kitti(imdb):
    def __init__(self, image_set, data_path, mc):
        imdb.__init__(self, 'kitti_'+image_set, mc)
        self._image_set = image_set
        self._data_root_path = data_path
        self._image_path = os.path.join(self._data_root_path, 'training', 'image_2')
        self._label_path = os.path.join(self._data_root_path, 'training', 'label_2')
        self._classes = self.mc.CLASS_NAMES
        self._class_to_idx = dict(zip(self.classes, xrange(self.num_classes)))
        print("self.num_classes:", self.num_classes)
        print("self.classes:", self.classes)
        print("class to idx:", self._class_to_idx)
    
        # a list of string indices of images in the directory
        self._image_idx = self._load_image_set_idx() 
        print("image_class:", self._image_idx)
    # a dict of image_idx -> [[cx, cy, w, h, cls_idx]]. x,y,w,h are not divided by
    # the image width and height
        self._rois = self._load_kitti_annotation()

    ## batch reader ##
        self._perm_idx = None
        self._cur_idx = 0
    # TODO(bichen): add a random seed as parameter
        self._shuffle_image_idx()
        self._eval_tool = './dataset/kitti-eval/cpp/evaluate_object'
        
    def _load_image_set_idx(self):
        image_set_file = os.path.join(
            self._data_root_path, 'ImageSets', self._image_set+'.txt')
        assert os.path.exists(image_set_file), \
            'File does not exist: {}'.format(image_set_file)

        with open(image_set_file) as f:
            image_idx = [x.strip() for x in f.readlines()]
        return image_idx

    def _image_path_at(self, idx):
        image_path = os.path.join(self._image_path, idx+'.png')
        assert os.path.exists(image_path), \
            'Image does not exist: {}'.format(image_path)
        return image_path

    def _load_kitti_annotation(self):
        def _get_obj_level(obj):
            height = float(obj[7]) - float(obj[5]) + 1
            truncation = float(obj[1])
            occlusion = float(obj[2])
            if height >= 40 and truncation <= 0.15 and occlusion <= 0:
                return 1
            elif height >= 25 and truncation <= 0.3 and occlusion <= 1:
                return 2
            elif height >= 25 and truncation <= 0.5 and occlusion <= 2:
                return 3
            else:
                return 4

        idx2annotation = {}
        for index in self._image_idx:
            filename = os.path.join(self._label_path, index+'.txt')
            with open(filename, 'r') as f:
                lines = f.readlines()
            f.close()
            bboxes = []
            for line in lines:
                obj = line.strip().split(' ')
                try:
                    cls = self._class_to_idx[obj[0].lower().strip()]
                except:
                    continue

                if self.mc.EXCLUDE_HARD_EXAMPLES and _get_obj_level(obj) > 3:
                    continue
                xmin = float(obj[4])
                ymin = float(obj[5])
                xmax = float(obj[6])
                ymax = float(obj[7])
                assert xmin >= 0.0 and xmin <= xmax, \
                   'Invalid bounding box x-coord xmin {} or xmax {} at {}.txt' \
                    .format(xmin, xmax, index)
                assert ymin >= 0.0 and ymin <= ymax, \
                   'Invalid bounding box y-coord ymin {} or ymax {} at {}.txt' \
                    .format(ymin, ymax, index)
                x, y, w, h = bbox_transform_inv([xmin, ymin, xmax, ymax])
                bboxes.append([x, y, w, h, cls])

            idx2annotation[index] = bboxes

        return idx2annotation

    def evaluate_detections(self, eval_dir, global_step, all_boxes):
        """Evaluate detection results.
        Args:
        eval_dir: directory to write evaluation logs
        global_step: step of the checkpoint
        all_boxes: all_boxes[cls][image] = N x 5 arrays of 
        [xmin, ymin, xmax, ymax, score]
        Returns:
        aps: array of average precisions.
        names: class names corresponding to each ap
        """
        det_file_dir = os.path.join(
             eval_dir, 'detection_files_{:s}'.format(global_step), 'data')
        if not os.path.isdir(det_file_dir):
            os.makedirs(det_file_dir)

        for im_idx, index in enumerate(self._image_idx):
            filename = os.path.join(det_file_dir, index+'.txt')
            with open(filename, 'wt') as f:
                for cls_idx, cls in enumerate(self._classes):
                    dets = all_boxes[cls_idx][im_idx]
                    for k in xrange(len(dets)):
                        f.write(
                            '{:s} -1 -1 0.0 {:.2f} {:.2f} {:.2f} {:.2f} 0.0 0.0 0.0 0.0 0.0 '
                            '0.0 0.0 {:.3f}\n'.format(
                                cls.lower(), dets[k][0], dets[k][1], dets[k][2], dets[k][3],
                                dets[k][4])
                        )

        cmd = self._eval_tool + ' ' \
              + os.path.join(self._data_root_path, 'training') + ' ' \
              + os.path.join(self._data_root_path, 'ImageSets',
                             self._image_set+'.txt') + ' ' \
              + os.path.dirname(det_file_dir) + ' ' + str(len(self._image_idx))

        print('Running: {}'.format(cmd))
        status = subprocess.call(cmd, shell=True)

        aps = []
        names = []
        for cls in self._classes:
            det_file_name = os.path.join(
                os.path.dirname(det_file_dir), 'stats_{:s}_ap.txt'.format(cls))
            if os.path.exists(det_file_name):
                with open(det_file_name, 'r') as f:
                    lines = f.readlines()
                assert len(lines) == 3, \
                   'Line number of {} should be 3'.format(det_file_name)

                aps.append(float(lines[0].split('=')[1].strip()))
                aps.append(float(lines[1].split('=')[1].strip()))
                aps.append(float(lines[2].split('=')[1].strip()))
            else:
                aps.extend([0.0, 0.0, 0.0])

            names.append(cls+'_easy')
            names.append(cls+'_medium')
            names.append(cls+'_hard')

        return aps, names

    def do_detection_analysis_in_eval(self, eval_dir, global_step):
        det_file_dir = os.path.join(
            eval_dir, 'detection_files_{:s}'.format(global_step), 'data')
        det_error_dir = os.path.join(
            eval_dir, 'detection_files_{:s}'.format(global_step),
            'error_analysis')
        if not os.path.exists(det_error_dir):
            os.makedirs(det_error_dir)
        det_error_file = os.path.join(det_error_dir, 'det_error_file.txt')

        stats = self.analyze_detections(det_file_dir, det_error_file)
        ims = self.visualize_detections(
            image_dir=self._image_path,
            image_format='.png',
            det_error_file=det_error_file,
            output_image_dir=det_error_dir,
            num_det_per_type=10
        )

        return stats, ims

    def analyze_detections(self, detection_file_dir, det_error_file):
        def _save_detection(f, idx, error_type, det, score):
            f.write(
                '{:s} {:s} {:.1f} {:.1f} {:.1f} {:.1f} {:s} {:.3f}\n'.format(
                    idx, error_type,
                    det[0]-det[2]/2., det[1]-det[3]/2.,
                    det[0]+det[2]/2., det[1]+det[3]/2.,
                    self._classes[int(det[4])], 
                    score
                    )
             )

    # load detections
        self._det_rois = {}
        for idx in self._image_idx:
            det_file_name = os.path.join(detection_file_dir, idx+'.txt')
            with open(det_file_name) as f:
                lines = f.readlines()
            f.close()
            bboxes = []
            for line in lines:
                obj = line.strip().split(' ')
                cls = self._class_to_idx[obj[0].lower().strip()]
                xmin = float(obj[4])
                ymin = float(obj[5])
                xmax = float(obj[6])
                ymax = float(obj[7])
                score = float(obj[-1])

                x, y, w, h = bbox_transform_inv([xmin, ymin, xmax, ymax])
                bboxes.append([x, y, w, h, cls, score])
            bboxes.sort(key=lambda x: x[-1], reverse=True)
            self._det_rois[idx] = bboxes

    # do error analysis
        num_objs = 0.
        num_dets = 0.
        num_correct = 0.
        num_loc_error = 0.
        num_cls_error = 0.
        num_bg_error = 0.
        num_repeated_error = 0.
        num_detected_obj = 0.

        with open(det_error_file, 'w') as f:
            for idx in self._image_idx:
                gt_bboxes = np.array(self._rois[idx])
                num_objs += len(gt_bboxes)
                detected = [False]*len(gt_bboxes)

                det_bboxes = self._det_rois[idx]
                if len(gt_bboxes) < 1:
                    continue

                for i, det in enumerate(det_bboxes):
                    if i < len(gt_bboxes):
                        num_dets += 1
                    ious = batch_iou(gt_bboxes[:, :4], det[:4])
                    max_iou = np.max(ious)
                    gt_idx = np.argmax(ious)
                    if max_iou > 0.1:
                        if gt_bboxes[gt_idx, 4] == det[4]:
                            if max_iou >= 0.5:
                                if i < len(gt_bboxes):
                                    if not detected[gt_idx]:
                                        num_correct += 1
                                        detected[gt_idx] = True
                                    else:
                                        num_repeated_error += 1
                            else:
                                if i < len(gt_bboxes):
                                    num_loc_error += 1
                                    _save_detection(f, idx, 'loc', det, det[5])
                        else:
                            if i < len(gt_bboxes):
                                num_cls_error += 1
                                _save_detection(f, idx, 'cls', det, det[5])
                    else:
                        if i < len(gt_bboxes):
                            num_bg_error += 1
                            _save_detection(f, idx, 'bg', det, det[5])

                for i, gt in enumerate(gt_bboxes):
                    if not detected[i]:
                        _save_detection(f, idx, 'missed', gt, -1.0)
                num_detected_obj += sum(detected)
        f.close()

        print ('Detection Analysis:')
        print ('    Number of detections: {}'.format(num_dets))
        print ('    Number of objects: {}'.format(num_objs))
        print ('    Percentage of correct detections: {}'.format(
            num_correct/num_dets))
        print ('    Percentage of localization error: {}'.format(
            num_loc_error/num_dets))
        print ('    Percentage of classification error: {}'.format(
            num_cls_error/num_dets))
        print ('    Percentage of background error: {}'.format(
            num_bg_error/num_dets))
        print ('    Percentage of repeated detections: {}'.format(
            num_repeated_error/num_dets))
        print ('    Recall: {}'.format(
            num_detected_obj/num_objs))

        out = {}
        out['num of detections'] = num_dets
        out['num of objects'] = num_objs
        out['% correct detections'] = num_correct/num_dets
        out['% localization error'] = num_loc_error/num_dets
        out['% classification error'] = num_cls_error/num_dets
        out['% background error'] = num_bg_error/num_dets
        out['% repeated error'] = num_repeated_error/num_dets
        out['% recall'] = num_detected_obj/num_objs

        return out

In [6]:
imdb = kitti('train', '../data/KITTI', mc)

self.num_classes: 3
self.classes: ['car', 'pedestrian', 'cyclist']
class to idx: {'cyclist': 2, 'car': 0, 'pedestrian': 1}
image_class: ['000001', '000002', '000005', '000007', '000009', '000006', '000004', '000008', '000003', '000000']


<table><tr><td bgcolor=orange > <font face="purisa" size=8 color='0000ff'>
 Extended learning
</font></td></tr></table>
<br>
<table><tr><td bgcolor=white > <font face="sawasdee" size=6 color='0000ff'>
range vs xrange
</font></td></tr></table>
<br>
## For the most part, xrange and range are the exact same in terms of functionality. They both provide a way to generate a list of integers for you to use, however you please. The only difference is that range returns a Python list object and xrange returns an xrange object.

In [8]:
a = xrange(0,10,2)
print("a:", a)
print("type of a:", type(a))
print("member of a:", "a[0]=", a[0], "a[1]=", a[1])
b = range(0,10,2)
print("b:", b)
print("type of b:", type(b))
print("member of b:","b[0]=", b[0],"b[1]=", b[1])

c = range(3)
print("c:", c)

a: xrange(0, 10, 2)
type of a: <type 'xrange'>
member of a: a[0]= 0 a[1]= 2
b: [0, 2, 4, 6, 8]
type of b: <type 'list'>
member of b: b[0]= 0 b[1]= 2
c: [0, 1, 2]


<table><tr><td bgcolor=orange > <font face="purisa" size=8 color='0000ff'>
 Extended learning
</font></td></tr></table>
<br>
<table><tr><td bgcolor=white > <font face="sawasdee" size=6 color='0000ff'>
zip
</font></td></tr></table>
<br>
## zip takes n number of iterables and returns list of tuples. ith element of the tuple is created using the ith element from each of the iterables.

In [10]:
list_a = [1, 2, 3, 4, 5]
list_b = ['a', 'b', 'c', 'd', 'e']

zipped_list = zip(list_a, list_b)

print (zipped_list)

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd'), (5, 'e')]


In [11]:
l = ['鹅鹅鹅', '曲项向天歌', '锄禾日当午', '春种一粒粟']

for i in l:
    print(i)

# 可以获取下表，enumerate每次循环可以得到下表及元素
for i, v in enumerate(l):
    print(i, v)

鹅鹅鹅
曲项向天歌
锄禾日当午
春种一粒粟
0 鹅鹅鹅
1 曲项向天歌
2 锄禾日当午
3 春种一粒粟


In [18]:
d = {'a':'apple', 'b':'banana', 'c':'car', 'd': 'desk'}

for key in d:
    # 遍历字典时遍历的是键
    print(key, d.get(key))

# for key, value in d.items():
# 上下两种方式等价 d.items() <=> dict.items(d)
for key, value in dict.items(d):
    print(key, value)

print([i for i in range(1, 11)])
print([i*2 for i in range(1, 11)])
print([i*i for i in range(1, 11)])
print([str(i) for i in range(1, 11)])
print([i for i in range(1, 11) if i % 2 == 0])

a apple
c car
b banana
d desk
a apple
c car
b banana
d desk
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
[2, 4, 6, 8, 10]


<table><tr><td bgcolor=orange > <font face="purisa" size=8 color='0000ff'>
 Extended learning
</font></td></tr></table>
<br>
<table><tr><td bgcolor=white > <font face="sawasdee" size=6 color='0000ff'>
strip
</font></td></tr></table>
<br>
## The strip() removes characters from both left and right based on the argument (a string specifying the set of characters to be removed).

In [25]:
b = 'rabc\t\n'
#print(b.strip()) # Remove '\n', '\r',  '\t',  ' '
print(b.strip('r'))
string = ' xoxo love xoxo   '

# Leading whitepsace are removed
print(string.strip())

print(string.strip(' o'))

# Argument doesn't contain space
# No characters are removed.
print(string.strip('sti'))

string = 'android is awesome'
print(string.strip('an'))

xoxo love xoxo
xoxo love xox
 xoxo love xoxo   
droid is awesome


<table><tr><td bgcolor=orange > <font face="purisa" size=8 color='0000ff'>
 Extended learning
</font></td></tr></table>
<br>
<table><tr><td bgcolor=white > <font face="sawasdee" size=6 color='0000ff'>
lower
</font></td></tr></table>
<br>
## The method lower() returns a copy of the string in which all case-based characters have been lowercased.